In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

# Load the JSON data
with open('netstats-315-326.json') as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
df = pd.DataFrame(data)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import random

### Helper functions

In [ ]:
def fill_blanks(df, srl_num, range_df, date_variable):
    """
    Fills missing observations for time series data.
    
    Parameters:
    - df: DataFrame containing the data to be processed.
    - srl_num: The column name in df that contains the series identifiers.
    - range_df: DataFrame containing the complete range of dates.
    - date_variable: The column name in both df and range_df that contains the date information.
    
    Returns:
    - DataFrame with missing observations filled.
    """
    filled_dfs = []  # List to hold the filled DataFrames for each unique series identifier

    for comb in df[srl_num].unique():
        #print('Processing series:', comb)
        temp = df[df[srl_num] == comb].copy()
        temp2 = range_df.merge(temp, how='left', on=date_variable)
        # Forward fill and then back fill to cover all missing values
        temp2.fillna(method='ffill', inplace=True)
        temp2.fillna(method='bfill', inplace=True)
        filled_dfs.append(temp2)

    # Concatenate all filled DataFrames
    filled_df = pd.concat(filled_dfs, ignore_index=True)

    return filled_df

### Dataset clean up

In [ ]:
# Remove ".os.net.ibm.com" from all host names
df['host'] = df['host'].str.replace('.os.net.ibm.com', '', regex=False)

# Identify columns where all values are zero
zero_only_columns = [col for col in df.columns if (df[col] == 0).all()]
packets_columns = [col for col in df.columns if 'packets' in col]
dropped_columns = [col for col in df.columns if'dropped' in col]
single_value_columns = list(df.columns[df.nunique() == 1])

# combine
columns_to_drop = list(set(zero_only_columns + packets_columns + dropped_columns + single_value_columns))

# Drop these columns from the DataFrame
df_cleaned = df.drop(columns=columns_to_drop)

# Ensure 'ts' column is in datetime format
df_cleaned['ts'] = pd.to_datetime(df_cleaned['ts'])

df_cleaned['ts'] = df_cleaned['ts'].dt.floor('T') 

# create unique column - serialize
df_cleaned['site_host'] = df_cleaned['site'] + "_" + df_cleaned['host']

# sort
df_cleaned.sort_values(by=['site_host','ts'],inplace=True)

In [ ]:
del(df)

In [ ]:
df_cleaned = df_cleaned.drop_duplicates()

In [ ]:
date_range = df_cleaned.ts.drop_duplicates()
date_range = pd.DataFrame(date_range)

In [ ]:
df_ready = fill_blanks(df_cleaned,'site_host',date_range,'ts')

In [ ]:
# Display the first few rows to understand the data structure
df_ready.head()

In [ ]:
del(df_cleaned)

### Get number of unique hosts / interface / and combination of the two

In [ ]:
print(f"Number of unique hosts: {len(df_ready.site.unique())}")
print(f"Number of unique interfaces: {len(df_ready.host.unique())}")

**Check for equal # of observations** <br>
accross all site /host combinatipons

In [ ]:
# Count of records per site/host comb
counts_per_site_host = df_ready.groupby(['site', 'host']).size().reset_index(name='count')

# looks for a single 
for cnt in counts_per_site_host['count'].unique():
    print(counts_per_site_host[counts_per_site_host['count'] == cnt].head(1))

After fill_blanks function all sites-host, combination have same number of observations

In [ ]:
# Display basic information about the DataFrame
df_ready.info()

In [ ]:
df_ready.to_csv('netstats_4_2.csv')